<a href="https://colab.research.google.com/github/vladkulikov/HelloWorldNC/blob/main/Code/Notebooks/enc_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
from typing import Tuple, List


''' Класс нормировки данных '''
class Normalizer():
    
    '''
    Parameters
    ----------
    dim : int
        Размерность входных данных.
    irr_dim : int
        Количество незначащих переменных.
    range_val_pairs : List[Tuple[float,float]]
        Диапазон значений входных данных.
    norm_min : float, optional
        Нижняя граница нормированных данных (по умолчанию 0).
    norm_max : float, optional
        Верхняя граница нормированных данных (по умолчанию 1).
    '''
    def __init__(self, dim : int, irr_dim : int, range_val_pairs : List[Tuple[float,float]], norm_min : float = 0., norm_max : float = 1.):
        self.dim = dim
        self.irr_dim = irr_dim
        self.range_val_pairs = range_val_pairs
        self.range_val = [(i[1] - i[0]) for i in self.range_val_pairs]
        self.norm_min = norm_min
        self.norm_max = norm_max
        self.range_norm = norm_max - norm_min
    
    
    def __normire(self, entryVal : float, ind : int) -> float:
        '''
        Parameters
        ----------
        entryVal : float
            Входное значение для нормировки.
        ind : int
            Индекс элемента в массиве.

        Returns
        -------
        float
            Нормированное значение.

        '''
        return self.norm_min + ((entryVal - self.range_val_pairs[ind][0]) * self.range_norm / self.range_val[ind])
    
    def __renormire(self, normVal : float, ind : int) -> float:
        '''
        Parameters
        ----------
        normVal : float
            Нормированное значение для денормировки.
        ind : int
            Индекс элемента в массиве.

        Returns
        -------
        float
            Денормированное значение.
        '''
        return self.range_val_pairs[ind][0] + ((normVal - self.norm_min) / self.range_norm * self.range_val[ind])
    
    def normalize(self, data) -> List[List[float]]:
        '''
        Parameters
        ----------
        data : list, array
            Входной набор данных для нормировки.

        Returns
        -------
        List[List[float]]
            Нормированный набор данных.
        '''
        count = 0
        if type(data) == list: 
          count = len(data)
        else:
          count = data.shape[0]
        normData = []
        for i in range(count):
            cur_sample = []
            for j in range(self.dim):
                cur_sample.append(self.__normire(data[i][j], j))
            for j in range(self.dim, self.dim + self.irr_dim):
                cur_sample.append(data[i][j])
            normData.append(cur_sample)
        return normData
    
    def renormalize(self, normData) -> List[List[float]]:
        '''
        Parameters
        ----------
        normData : list, array
            Нормированный набор данных.

        Returns
        -------
        List[List[float]]
            Денормированный набор данных.
        '''
        count = 0
        if type(normData) == list: 
          count = len(normData)
        else:
          count = normData.shape[0]
        data = []
        for i in range(count):
            cur_sample = []
            for j in range(self.dim):
                cur_sample.append(self.__renormire(normData[i][j], j))
            for j in range(self.dim, self.dim + self.irr_dim):
                cur_sample.append(normData[i][j])
            data.append(cur_sample)
        return data


In [34]:
import numpy as np
import random
import pandas as pd
!pip install git+https://github.com/naught101/sobol_seq@v0.2.0#egg=sobol_seq
import sobol_seq
import time
from typing import Tuple, List


''' Класс генерации данных '''
class DataGenerator():

    '''
    Parameters
    ----------
    dim: int
        Размерность входных данных.
    val_range : List[Tuple[float,float]]
        Диапазон значений входных данных.
    '''
    def __init__(self, dim : int, val_range : List[Tuple[float,float]]):
        try:
            assert dim  == len(val_range), 'Размерность входных диапазонов не равна входной размерности!'
            self.dim = dim
            self.val_range = val_range
            random.seed(int(time.time()))
        except AssertionError as e:
            raise AssertionError(e.args[0])
    

    def get_random(self, samples_num : int, irrelevant_var_count : int = 0, write_in_file : bool = False) -> List[List[float]]:
        '''
        Рандомная генерация данных.
        Parameters
        ----------
        samples_num : int
            Количество записей.
        irrelevant_var_count : int, optional
            Количество незначащих переменных. По умолчанию 0.
        write_in_file : bool, optional
            Запись в файл. По умолчанию False.

        Returns
        -------
        List[List[float]]
            Список записей.
        '''
        arr = []
        for k in range(samples_num):
            sample = []
            # добавляем существенные переменные
            for i in self.val_range:
                sample.append(random.uniform(i[0], i[1]))
            # добавляем несущественные переменные
            if irrelevant_var_count != 0:
                for i in range(irrelevant_var_count):
                    sample.append(random.uniform(0., 1.))
            arr.append(sample)
        if write_in_file:
            col = [('x' + str(i+1)) for i in range(self.dim + irrelevant_var_count)]
            df = pd.DataFrame(arr, columns=col)
            df.to_csv(f'../../DataSet/random_{self.dim}_{samples_num}_{irrelevant_var_count}.csv')
        return arr
    

    def get_sobol(self, samples_num : int, irrelevant_var_count : int = 0, write_in_file : bool = False) -> List[List[float]]:
        '''
        Генерация данных от 0 до 1 методом Sobol.
        Parameters
        ----------
        samples_num : int
            Количество записей.
        irrelevant_var_count : int, optional
            Количество незначащих переменных. По умолчанию 0.
        write_in_file : bool, optional
            Запись в файл. По умолчанию False.

        Returns
        -------
        List[List[float]]
            Список записей.
        '''
        arr = sobol_seq.i4_sobol_generate(self.dim, samples_num)
        if irrelevant_var_count != 0:
            zeros = [[0] for i in range(irrelevant_var_count)]
            arr = np.insert(arr, obj=self.dim, values=zeros, axis=1)
        if write_in_file:
            col = [('x' + str(i+1)) for i in range(self.dim + irrelevant_var_count)]
            df = pd.DataFrame(arr, columns=col)
            df.to_csv(f'../DataSet/sobol_{self.dim}_{samples_num}_{irrelevant_var_count}.csv')
        return list(arr)
    
    
    def get_from_file(self, filename : str) -> List[List[float]]:
        '''
        Parameters
        ----------
        filename : str
            Имя файла.

        Raises
        ------
        OSError
            Файл не найден.

        Returns
        -------
        List[List[float]]
            Список записей.
        '''
        try:
            return list(pd.read_csv(filename, index_col=0).to_numpy('float32'))
        except OSError as e:
            raise OSError(e.args[0])

  Cloning https://github.com/naught101/sobol_seq (to revision v0.2.0) to /tmp/pip-install-9f7ivah_/sobol-seq_944d6928d11f485fbdd577634b495a02
  Running command git clone -q https://github.com/naught101/sobol_seq /tmp/pip-install-9f7ivah_/sobol-seq_944d6928d11f485fbdd577634b495a02
  Running command git checkout -q 8f819b68f64e1fea4999f5c64b73823a019fb244


In [35]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
import keras.backend as K
from keras.layers import Lambda
import numpy as np

''' Класс автоэнкодеров '''
class AutoencoderClass():
  def __init__(self, func, input_dim : int, encoding_dim : int, activations : list, enc_type : str, normalizer : Normalizer):
    self.func = func                 # Функция обучения
    self.batch = 0                   # Размр батча
    self.input_dim = input_dim       # Размерность входного представления
    self.encoding_dim = encoding_dim # Размерность кодированного представления
    self.activations = activations   # Функции активации
    self.enc_type = enc_type         # Тип автоэнкодера
    self.aes_types = {'dense': self.__create_dense_ae,
                      'deep':  self.__create_deep_dense_ae,
                      'conv':  self.__create_deep_conv_ae,
                      'vae':   self.__create_vae}
    self.normalizer = normalizer
    try:
      # Сборка моделей
      self.encoder, self.decoder, self.autoencoder = self.aes_types[self.enc_type]()
      if self.aes_types != 'vae':
        self.autoencoder.compile(optimizer = 'adam', loss = self.custom_loss, metrics=['accuracy'])
      else:
        self.autoencoder.compile(optimizer = 'adam', loss = self.vae_loss, metrics=['accuracy'])
    except KeyError as e:
      raise ValueError('Undefined unit: {}'.format(e.args[0]))

  # Обучение модели
  def fit(self, train_data, test_data, epochs : int, batch_size : int, shuffle : bool):
    self.batch = batch_size
    if self.enc_type != 'conv':
      self.autoencoder.fit(train_data, train_data,
                           epochs=epochs,
                           batch_size=self.batch,
                           shuffle=shuffle,
                           validation_data=(test_data, test_data))
    else:
      grid_train = []
      grid_test = []
      for i in range(len(train_data)):
        xx, yy = np.meshgrid(train_data[i], train_data[i])
        grid_train.append(xx)

      for i in range(len(test_data)):
        xx, yy = np.meshgrid(test_data[i], test_data[i])
        grid_test.append(xx)
      
      self.autoencoder.fit(grid_train, grid_train,
                           epochs=epochs,
                           batch_size=self.batch,
                           shuffle=shuffle,
                           validation_data=(grid_test, grid_test))

  # Предсказание результата
  def predict(self, x_vector):
    if self.enc_type != 'conv':
      return self.autoencoder.predict(x_vector)
    else:
      return self.autoencoder.predict(x_vector)[0]

  # Тип автоэнкодера
  def get_aec_type(self):
    return self.enc_type

  # Возвращает собранные модели
  def get_models(self):
    return self.autoencoder, self.encoder, self.decoder

  # Loss функция
  @tf.autograph.experimental.do_not_convert
  def custom_loss(self, x_true, x_pred):
    return K.mean(K.abs(self.func(self.normalizer.renormalize(x_pred)[0]) - self.func(self.normalizer.renormalize(x_true)[0])))

  # Loss функция для вариационного автоэнкодера
  @tf.autograph.experimental.do_not_convert
  def vae_loss(self, x_true, x_pred):
    x_true = K.reshape(x_true, shape=(batch_size, self.input_dim))
    x_pred = K.reshape(x_pred, shape=(batch_size, self.input_dim))
    loss = self.custom_loss(x_true, x_pred)
    kl_loss = -0.5 * K.sum(1 + self.z_log_var - K.square(self.z_mean) - K.exp(self.z_log_var))
    return loss + kl_loss

  ''' Сжимающий автоэнкодер '''
  def __create_dense_ae(self):
    # Энкодер
    input_data = Input(shape=(self.input_dim))
    encoded = Dense(self.encoding_dim, activation = self.activations[0])(input_data)
    
    # Декодер
    input_encoded = Input(shape = (self.encoding_dim))
    decoded = Dense(self.input_dim, activation = self.activations[1])(input_encoded)

    # Модели
    encoder = Model(input_data, encoded, name = "encoder")
    decoder = Model(input_encoded, decoded, name = "decoder")
    autoencoder = Model(input_data, decoder(encoder(input_data)), name = "autoencoder")
    return encoder, decoder, autoencoder

  ''' Глубокий автоэнкодер '''
  def __create_deep_dense_ae(self):
    # Энкодер
    input_data = Input(shape=(self.input_dim))
    x = Dense(self.encoding_dim*2, activation='relu')(input_data)
    encoded = Dense(self.encoding_dim, activation='linear')(x)
    
    # Декодер
    input_encoded = Input(shape=(self.encoding_dim,))
    x = Dense(self.encoding_dim*2, activation='relu')(input_encoded)
    decoded = Dense(self.input_dim, activation='sigmoid')(x)
    
    # Модели
    encoder = Model(input_data, encoded, name="encoder")
    decoder = Model(input_encoded, decoded, name="decoder")
    autoencoder = Model(input_data, decoder(encoder(input_data)), name="autoencoder")
    return encoder, decoder, autoencoder

  ''' Сверточный автоэнкодер '''
  def __create_deep_conv_ae(self):
    # Энкодер
    input_data = Input(shape=(self.input_dim, self.input_dim, 1))
    x = Conv2D(25, (2, 2), activation='relu', padding='same')(input_data)
    x = MaxPooling2D((2, 2), padding='same')(x)
    #x = Conv2D(32, (2, 2), activation='relu', padding='same')(x)
    #x = MaxPooling2D((2, 2), padding='same')(x)
    encoded = Conv2D(1, (2, 2), activation='relu', padding='same')(x)

    # На этом моменте представление  (7, 7, 1) т.е. 49-размерное
    
    # Декодер
    input_encoded = Input(shape=(7, 7, 1))
    #x = Conv2D(32, (7, 7), activation='relu', padding='same')(input_encoded)
    #x = UpSampling2D((2, 2))(x)
    x = Conv2D(25, (2, 2), activation='relu', padding='same')(input_encoded)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (2, 2), activation='sigmoid', padding='same')(x)

    # Модели
    encoder = Model(input_data, encoded, name="encoder")
    decoder = Model(input_encoded, decoded, name="decoder")
    autoencoder = Model(input_data, decoder(encoder(input_data)), name="autoencoder")
    return encoder, decoder, autoencoder

  ''' Вариационный автоэнкодер                         '''
  ''' Работает на основе девергенции Кульбака-Лейблера '''
  ''' Идея: переход данных скрытого слоя к нормальному распределению'''
  ''' Статья: https://habr.com/ru/post/484756/ '''
  ''' Видео:  https://youtu.be/ebI3JLAcWqQ '''
  def __create_vae(self):
    hidden_dim = 2

    input_data = Input(shape=(self.input_dim))
    x = Dense(self.encoding_dim, activation='relu')(input_data)
    
    self.z_mean = Dense(self.encoding_dim)(x)    # Мат ожидание
    self.z_log_var = Dense(self.encoding_dim)(x) # Логарифм дисперсии
    
    # Нормальное распределение N(0, 1)
    def noiser(args):
      self.z_mean, self.z_log_var = args
      N = K.random_normal(shape=(self.batch, self.encoding_dim), mean=0., stddev=1.0)
      return K.exp(self.z_log_var / 2) * N + self.z_mean
    
    # Преобразование данных в нормальное распределения
    h = Lambda(noiser, output_shape=(self.encoding_dim,))([self.z_mean, self.z_log_var])
    
    input_encoded = Input(shape=(self.encoding_dim,))
    d = Dense(self.encoding_dim, activation='relu')(input_encoded)
    decoded = Dense(self.input_dim, activation='sigmoid')(d)
    
    encoder = Model(input_data, h, name='encoder')
    decoder = Model(input_encoded, decoded, name='decoder')
    vae = Model(input_data, decoder(encoder(input_data)), name="vae")
    return encoder, decoder, vae

In [45]:
import numpy as np
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

def func(x):
  return tf.math.sin(x[0]) + tf.math.sin(x[1]) + tf.math.sin(x[2]) + tf.math.sin(x[3]) + tf.math.sin(x[4]) + tf.math.sin(x[5])

def disp_res(orig_x, orig_y, pred_x, pred_y):
  n = len(orig_x)
  for i in range(n):
    print(f'Orig X: {orig_x[i]}')
    print(f'Orig Y: {orig_y[i]}')
    print(f'Pred X: {pred_x[i]}')
    print(f'Pred Y: {pred_y[i]}\n')


def compare(orig_data, pred_data):
  # clf = svm.SVC(kernel='linear', C=1, random_state=42)
  # scores_x = cross_val_score(clf, orig_data[0:10], pred_data[0:10], cv=5)
  # scores_y = cross_val_score(clf, [func(x) for x in orig_data][0:10], [func(x) for x in pred_data][0:10], cv=5)
  y_orig = [func4(x) for x in orig_data]
  y_pred = [func4(x) for x in pred_data]
  k = 10
  disp_res(orig_data[0:k], y_orig[0:k], pred_data[0:k], y_pred[0:k])

  x_error = mean_absolute_error(orig_data, pred_data)
  y_error = mean_absolute_error(y_orig, y_pred)

  print(f'Abs error X: {x_error:.2f}')
  print(f'Abs error Y: {y_error:.2f}')

In [46]:
 #данные, тип автоэнкодера, шаг эпох, шаг батчей, шаг размерности, исходная размерность, размерность незначимых, шаг разбиения на выборку, количество точек
import sys
from numpy import arange
def searchOpt(sobol_data ,enc_type : str, h_epoch : int, h_batch : int, h_size : int, dim : int, irr_dim : int, h_percent : float, n : int):
  error = sys.float_info.max
  hp_list = list()
  for epoch in range(5, 60, h_epoch):
    for batch in range(4, 256, h_batch):
      for size in range(dim // 2, dim ,h_size):
        for percent in arange(0.5, 1.0, h_percent):
            data_train = np.array(sobol_data[0:int(n * h_percent)])
            data_test = np.array(sobol_data[int(n * h_percent):n])
            model = AutoencoderClass(func, dim + irr_dim, size, list(['relu', 'sigmoid']), enc_type, normalizer)
            model.fit(data_train, data_test, epoch, batch, True)
            rand_data = generator.get_random(100)
            pred_data = normalizer.renormalize([model.predict(np.array(x).reshape(1,dim + irr_dim))[0] for x in normalizer.normalize(rand_data)])
            if error > compare(rand_data, pred_data):
              error = compare(rand_data, pred_data)
              hp_list.clear()
              hp_list.append(enc_type)
              hp_list.append(epoch)
              hp_list.append(batch)
              hp_list.append(size)
              hp_list.append(percent)
  return hp_list

In [47]:

def func4(x):
  return x1(x[0]) + x2(x[0]) + x3(x[0]) + 2 * x4(x[0]) + x5(x[0])

def x1(x):
  return x

def x2(x):
  return tf.math.sin(x) * tf.math.sin(x)

def x3(x):
  return tf.math.cos(x) * tf.math.cos(x)

def x4(x):
  return x * x

def x5(x):
  return 2 * x - 1

In [51]:
if __name__ == "__main__":


  dim = 1
  irr_dim = 0
  data_range = [(0, math.pi / 2)]
  generator = DataGenerator(dim, data_range)
  normalizer = Normalizer(dim, irr_dim, data_range)

  n = 10000
  sobol_data = generator.get_sobol(n, irr_dim)
  random.shuffle(sobol_data)
  data_train = np.array(sobol_data[0:int(n * 0.8)])
  data_test = np.array(sobol_data[int(n * 0.8):n])
  model = AutoencoderClass(func4, dim + irr_dim, 5, list(['relu', 'sigmoid']), 'vae', normalizer)
  model.fit(data_train, data_test, 15, 10, True)

  rand_data = generator.get_random(100, irr_dim)
  pred_data = normalizer.renormalize([model.predict(np.array(x).reshape(1,dim + irr_dim))[0] for x in normalizer.normalize(rand_data)])
  #pred_data = normalizer.renormalize([model.predict(x.reshape(1,dim + irr_dim))[0] for x in sobol_data[0:100]])
  #compare(normalizer.renormalize(sobol_data)[0:100], pred_data[0:100])
  #searchOpt(sobol_data, 'vae', 2, 8, 1, dim, irr_dim, 0.1, n)
  compare(rand_data, pred_data)

Epoch 1/15
800/800 [==============================] - 3s 2ms/step - loss: 2.4816 - accuracy: 0.0000e+00 - val_loss: 2.3672 - val_accuracy: 0.0000e+00
Epoch 2/15
800/800 [==============================] - 1s 2ms/step - loss: 2.2390 - accuracy: 0.0000e+00 - val_loss: 2.2335 - val_accuracy: 0.0000e+00
Epoch 3/15
800/800 [==============================] - 1s 2ms/step - loss: 2.0413 - accuracy: 0.0000e+00 - val_loss: 1.8754 - val_accuracy: 0.0000e+00
Epoch 4/15
800/800 [==============================] - 1s 2ms/step - loss: 1.5862 - accuracy: 0.0000e+00 - val_loss: 1.3925 - val_accuracy: 0.0000e+00
Epoch 5/15
800/800 [==============================] - 1s 2ms/step - loss: 1.1151 - accuracy: 0.0000e+00 - val_loss: 0.9913 - val_accuracy: 0.0000e+00
Epoch 6/15
800/800 [==============================] - 1s 2ms/step - loss: 0.8566 - accuracy: 0.0000e+00 - val_loss: 0.6627 - val_accuracy: 0.0000e+00
Epoch 7/15
800/800 [==============================] - 1s 2ms/step - loss: 0.6559 - accuracy: 0.0000e